In [1]:
import os, sys, csv, json, re
import parseidf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Read Red Nodes and Mapping file

In [4]:
work_folder = 'san-francisco'
scene_folder = 'scene-json'

surf_id_to_name = {}
with open(os.path.join(scene_folder, 'meta', 'sf_dt_98_red_nodes_data_exchanges.json'), 'r') as f:
    eplus_meta = json.load(f)
    for bldg in eplus_meta['features']:
        bldg_id = bldg['id']
        for surf in bldg['properties']['objects']:
            surf_id = bldg_id + ':' + surf['id']
            surf_name = surf['properties']['name']
            surf_id_to_name[surf_id] = bldg_id + ':' + surf_name
len(surf_id_to_name)

3303

In [5]:
surf_name_to_wrf_id = {}
with open(os.path.join(scene_folder, 'meta', 'sf_dt_97_mapping.json'), 'r') as f_map:
    eplus_mapping = json.load(f_map)
    for bldg in eplus_mapping['features']:
        bldg_id = bldg['id']
        for surf in bldg['properties']['objects']:
            surf_id = bldg_id + ':' + surf['id']
            wrf_id = surf['properties']['cityffd_id']
            surf_name_to_wrf_id[surf_id_to_name[surf_id]] = wrf_id
            
len(surf_name_to_wrf_id)

3303

In [6]:
surf_name_to_wrf_id['bldg_3981165_1754:S2-office-core-wall-7']

2809

In [7]:
surf_name_to_wrf_id['bldg_3981165_1754:S6-office-core-roof']

2834

## Read CSV schedule for this round

In [8]:
SCHEDULE_FILE_NAME = 'sf_97_2m_hvac_cfd_r2.csv' ## todo
df_csv_result = pd.read_csv(os.path.join(scene_folder, "run_time_inputs", SCHEDULE_FILE_NAME), sep=',', encoding='UTF-8')
df_csv_result.head()

,drybulb:1,wind_speed:1,wind_dir:1,drybulb:2,wind_speed:2,wind_dir:2,drybulb:3,wind_speed:3,wind_dir:3,drybulb:4,...,wind_dir:3300,drybulb:3301,wind_speed:3301,wind_dir:3301,drybulb:3302,wind_speed:3302,wind_dir:3302,drybulb:3303,wind_speed:3303,wind_dir:3303
0,27.180,0.014,356.846,27.180,0.014,356.846,27.180,0.014,356.846,27.180,...,266.255,26.884,0.028,270.939,26.061,0.020,224.428,26.762,0.010,285.170
1,25.175,0.009,0.068,25.175,0.009,0.068,25.175,0.009,0.068,25.175,...,249.655,25.834,0.019,231.409,24.770,0.015,83.512,25.270,0.023,230.355
2,23.671,0.004,347.859,23.671,0.004,347.859,23.671,0.004,347.859,23.671,...,183.004,26.597,0.329,181.754,22.955,0.216,90.107,25.200,0.219,125.456
3,23.914,0.011,190.953,23.914,0.011,190.953,23.914,0.011,190.953,23.914,...,185.479,26.935,0.381,177.588,23.636,0.282,83.839,25.333,0.233,117.696
4,24.072,0.009,217.414,24.072,0.009,217.414,24.072,0.009,217.414,24.072,...,249.582,25.006,0.132,185.392,24.691,0.179,74.695,24.421,0.187,148.726


### map air node to CSV file name and column to write in IDF

In [10]:
header_to_column_num = {}
for i in range(34):
    header_to_column_num[i] = {}
    SCHEDULE_FILE_NAME = 'sf_97_2m_hvac_cfd_r2_' + str(i) + '.csv' ## todo
    df_csv_result = pd.read_csv(os.path.join(scene_folder, "run_time_inputs", SCHEDULE_FILE_NAME), sep=',', encoding='UTF-8')
    column_num = 2
    key_list = list(df_csv_result.keys())
    for key in key_list[1:]:
        header_to_column_num[i][key] = column_num
        column_num += 1
# len(header_to_column_num[i])

In [8]:
header_to_column_num[33]

{'drybulb:3301': 2,
 'wind_speed:3301': 3,
 'wind_dir:3301': 4,
 'drybulb:3302': 5,
 'wind_speed:3302': 6,
 'wind_dir:3302': 7,
 'drybulb:3303': 8,
 'wind_speed:3303': 9,
 'wind_dir:3303': 10}

## Write IDF files with local air nodes

In [11]:
WORK_FOLDER_NAME = 'SF_100'
BLDG_NAME_LIST = os.listdir(WORK_FOLDER_NAME)

for bldg_name in BLDG_NAME_LIST:
    input_file_path = os.path.join(WORK_FOLDER_NAME, bldg_name, 'model-1.idf')
    output_file_path= os.path.join(WORK_FOLDER_NAME, bldg_name, 'model-wrf-2.idf') ## TODO
    
    if not os.path.exists(input_file_path):
        continue
        
    ext_surfaces = list()
    ext_win_surfaces = {}
    
    with open(input_file_path, 'r') as f:
        idf = parseidf.parse(f.read())

        for surface_detailed in idf['BuildingSurface:Detailed'.upper()]:
            if surface_detailed[5] == 'Outdoors':
                ext_surfaces.append(surface_detailed[1])
        for surface_detailed in idf['FenestrationSurface:Detailed'.upper()]:
            ext_win_surfaces[surface_detailed[1]] = surface_detailed[4]

        idf['Schedule:File'.upper()] = list()

        for surf_name in ext_surfaces:
            for header in ['drybulb', 'wind_speed', 'wind_dir']:
                wrf_id = surf_name_to_wrf_id[bldg_name + ':' + surf_name.replace(' ', '-')]
                file_id = wrf_id // 100
                if wrf_id % 100 == 0:
                    file_id -= 1
                file_name = '../../scene-json/run_time_inputs/sf_97_2m_hvac_cfd_r2_' + str(file_id) + '.csv' ## TODO
                header_name = header + ':' + str(wrf_id)
                column_num = header_to_column_num[file_id][header_name]
                idf['Schedule:File'.upper()].append(['Schedule:File',
                                                     surf_name + ':' + header, 
                                                     'Any Number',
                                                     file_name, 
                                                     str(column_num), 
                                                     '1'])

        idf['SurfaceProperty:LocalEnvironment'.upper()] = list()
        idf['OutdoorAir:Node'.upper()] = list()

        for surf_name in ext_surfaces:
            idf['SurfaceProperty:LocalEnvironment'.upper()].append(['SurfaceProperty:LocalEnvironment',
                                                                    'LocEnv:' + surf_name, 
                                                                    surf_name, 
                                                                    '',
                                                                    '', 
                                                                    'OutdoorAirNode:' + surf_name])

            loc_node_list = ['OutdoorAir:Node', 
                             'OutdoorAirNode:' + surf_name, 
                             '',
                             surf_name + ':' + 'drybulb', 
                             '', 
                             surf_name + ':' + 'wind_speed', 
                             surf_name + ':' + 'wind_dir']
            idf['OutdoorAir:Node'.upper()].append(loc_node_list)
        
        for surf_name in ext_win_surfaces:
            base_surf_name = ext_win_surfaces[surf_name]
            idf['SurfaceProperty:LocalEnvironment'.upper()].append(['SurfaceProperty:LocalEnvironment',
                                                                    'LocEnv:' + surf_name, 
                                                                    surf_name, 
                                                                    '',
                                                                    '', 
                                                                    'OutdoorAirNode:' + base_surf_name])
            

    with open(output_file_path, 'w') as out:
        for idf_object_key in idf:
            idf_objects = idf[idf_object_key]
            for idf_object in idf_objects:
                for attr_num in range(len(idf_object)):
                    if attr_num == 0:
                        out.write('  ' + str(idf_object[attr_num]) + ',\n')
                    elif attr_num == len(idf_object) - 1:
                        out.write('    ' + str(idf_object[attr_num]) + ';\n')
                    else:
                        out.write('    ' + str(idf_object[attr_num]) + ',\n')
                out.write('\n')